In [2]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import numpy as np
import time

In [70]:
DataSource_df = pd.read_excel('竞品数据源.xlsx')
links_df = pd.DataFrame()
links_df['Original Links'] = DataSource_df['goods_id'].apply(lambda x: 'https://item.taobao.com/item.htm?id='+str(x))
links_df.drop_duplicates(inplace =True)

In [72]:
links_df.head(5)

,Original Links
0,https://item.taobao.com/item.htm?id=20565599246
1,https://item.taobao.com/item.htm?id=586375262300
2,https://item.taobao.com/item.htm?id=611391648091
3,https://item.taobao.com/item.htm?id=560263330969
4,https://item.taobao.com/item.htm?id=595492174761


In [6]:
browser = webdriver.Chrome()
# 打开淘宝登录先登录
browser.get("https://login.taobao.com/member/login.jhtml?spm=a21bo.2017.754894437.1.5af911d9q07jcU&f=top&redirectURL=https%3A%2F%2Fwww.taobao.com%2F")
# 定义js代码
script = "Object.defineProperty(navigator,'webdriver',{get: ()=> false,});"
# 执行js代码
browser.execute_script(script)#这样登录的时候就不会出现滑块

In [127]:
shopping_links_list =[]
shopping_final_links_list = []
title_list =[]
is_sold_out = []
underline_price_list = []
store_name_list= []
store_link_list = []
prmote_price_list = []
activity_price_list =[]
activity_info_list = []
activity_detail_info_list = []
coupon_info_list = []
sales_list = []
comment_num_list = []
platform_list = []

In [132]:
total_count = 0
count = 0
for link in links_df['Original Links'][80:]:
    browser.get(link)
    time.sleep(3)
    currentPageUrl = browser.current_url
    shopping_links_list.append(link)
    shopping_final_links_list.append(currentPageUrl)
    if currentPageUrl.find('tmall')>0:
        print('天猫链接')
        platform_list.append('tm')
#         tm_df = tm_df.append([{'链接':currentPageUrl,'web':'tm'}],ignore_index=True)
#天猫解析部分
        try:
            title = browser.find_element_by_xpath('//*[@id="J_DetailMeta"]/div[1]/div[1]/div/div[1]/h1').text#标题
        except NoSuchElementException:
            title='商品不存在'
        title_list.append(title)
        
        try:
            sold_out_flag = browser.find_element_by_xpath('//*[@id="J_Sold-out-recommend"]/div[1]/div[1]/strong').text#是否下架
            sold_out_flag = 1
        except NoSuchElementException:    
            sold_out_flag = 0
        is_sold_out.append(sold_out_flag)
        
        try:
            store_name = browser.find_element_by_xpath('//*[@id="shopExtra"]/div[1]/a').text#店铺名称
        except NoSuchElementException:
            store_name = '天猫超市'
        store_name_list.append(store_name)
        
        
        try:
            store_temp_link =  browser.find_element_by_xpath('//*[@id="shopExtra"]/div[1]/a').get_attribute("href")#店铺链接
            store_link =  'https:'+store_temp_link
        except NoSuchElementException:
            store_link = 'https://chaoshi.tmall.com/'
        store_link_list.append(store_link)
    


        try:
            underline_price = browser.find_element_by_xpath('//*[@id="J_StrPriceModBox"]/dd/span[1]').text#划线价
        except NoSuchElementException:
            underline_price = np.nan
        underline_price_list.append(underline_price)

        try:
            prmote_price = browser.find_element_by_xpath('//*[@id="J_PromoPrice"]/dd/div/span').text#促销价
        except NoSuchElementException:
            prmote_price = np.nan
        prmote_price_list.append(prmote_price)
      
        
        try:
            activity_price = browser.find_element_by_xpath('//*[@id="J_ActivityPrice"]').text#活动价
        except NoSuchElementException:
            activity_price =np.nan
        activity_price_list.append(activity_price)

        try:
            activity_info = browser.find_element_by_xpath('//*[@id="J_DetailMeta"]/div[1]/div[1]/div/div[2]/dl[3]/div[1]/dd').text#活动信息
        except NoSuchElementException:
            activity_info = np.nan
        activity_info_list.append(activity_info)

        try:
            browser.find_element_by_xpath('//*[@id="J_DetailMeta"]/div[1]/div[1]/div/div[2]/dl[3]/div[1]/dt').click()
            time.sleep(0.3)
            act_detail = browser.find_element_by_xpath('//*[@id="J_DetailMeta"]/div[1]/div[1]/div/div[2]/dl[3]/div[2]/div/ul').text#活动详细信息
        except NoSuchElementException:
            act_detail = np.nan
        except WebDriverException:
            act_detail = np.nan
        activity_detail_info_list.append(act_detail) 

        try:
            coupon_info = browser.find_element_by_xpath('//*[@id="J_DetailMeta"]/div[1]/div[1]/div/ul/li[3]/div/a').text#优惠券信息
        except NoSuchElementException:
            coupon_info =np.nan
        coupon_info_list.append(coupon_info)

        try:
            sales = browser.find_element_by_xpath('//*[@id="J_DetailMeta"]/div[1]/div[1]/div/ul/li[1]/div/span[2]').text#销量
        except NoSuchElementException:
            sales =np.nan
        sales_list.append(sales)

        try:
            comment_num = browser.find_element_by_xpath('//*[@id="J_ItemRates"]/div/span[2]').text#评论数
        except NoSuchElementException: 
            comment_num=np.nan
        comment_num_list.append(comment_num)

#######################################################################################################################################  
#淘宝解析部分
    else:
        platform_list.append('tb')
        try:
            title = browser.find_element_by_xpath('//*[@id="J_Title"]/h3').text#标题
        except NoSuchElementException:
            title='商品不存在'
        title_list.append(title)
        
        
        
        try:#尝试全球购解析
            store_name = browser.find_element_by_xpath('//*[@id="header-content"]/div[2]/div[1]/div[1]/a').text#店铺名称
        except NoSuchElementException:
            try:#尝试一般淘宝解析
                store_name = browser.find_element_by_xpath('//*[@id="J_ShopInfo"]/div/div[1]/div[1]').text#店铺名称
            except NoSuchElementException:
                store_name = np.nan
        store_name_list.append(store_name)
        
        try:
            sold_out_flag = browser.find_element_by_xpath('//*[@id="J_Sold-out-recommend"]/div[1]/div[1]/strong').text#是否下架
            sold_out_flag = 1
        except NoSuchElementException:    
            sold_out_flag = 0
        is_sold_out.append(sold_out_flag)
        
        try:#尝试全球购店铺链接
            store_link =  browser.find_element_by_xpath('//*[@id="header-content"]/div[2]/div[1]/div[1]/a').get_attribute("href")#店铺链接
        except NoSuchElementException:
            try:#尝试一般淘宝店铺链接
                store_link =  browser.find_element_by_xpath('//*[@id="J_ShopInfo"]/div/div[1]/div[1]/dl/dd/strong/a').get_attribute("href")#店铺链接
            except NoSuchElementException:
                store_link = np.nan
        store_link_list.append(store_link)
        
        try:
            underline_price = browser.find_element_by_xpath('//*[@id="J_StrPrice"]/em[2]').text#划线价
        except NoSuchElementException:
            underline_price = np.nan
        underline_price_list.append(underline_price)

        try:
            prmote_price = browser.find_element_by_xpath('//*[@id="J_PromoPriceNum"]').text#促销价
        except NoSuchElementException:
            prmote_price = np.nan
        prmote_price_list.append(prmote_price)

        try:
            activity_price = browser.find_element_by_xpath('//*[@id="J_ActivityPrice"]').text#活动价
        except NoSuchElementException:
            activity_price = np.nan
        activity_price_list.append(activity_price)

        try:
            activity_info = browser.find_element_by_xpath('//*[@id="J_PromoType"]').text#活动信息
        except NoSuchElementException:
            activity_info = np.nan
        activity_info_list.append(activity_info)

        try:
            browser.find_element_by_xpath('//*[@id="J_DetailMeta"]/div[1]/div[1]/div/div[2]/dl[3]/div[1]/dt').click()
            print('找到点击元素')
            time.sleep(0.3)
            act_detail = browser.find_element_by_xpath('//*[@id="J_DetailMeta"]/div[1]/div[1]/div/div[2]/dl[3]/div[2]/div/ul').text#活动详细信息
        except NoSuchElementException:
            act_detail = np.nan
        except WebDriverException:
            act_detail = np.nan
        activity_detail_info_list.append(act_detail)    

        try:
            coupon_info = browser.find_element_by_xpath('//*[@id="J_OtherDiscount"]/div').text#优惠券信息
        except NoSuchElementException:  
            coupon_info = np.nan
        coupon_info_list.append(coupon_info)

        try:
            sales = browser.find_element_by_xpath('//*[@id="J_SellCounter"]').text#销量
        except NoSuchElementException:
            sales = np.nan
        sales_list.append(sales)

        try:
            comment_num = browser.find_element_by_xpath('//*[@id="J_TabBar"]/li[2]/a/em').text#评论数
        except NoSuchElementException:
            comment_num = np.nan
        comment_num_list.append(comment_num)
        
    total_count =  total_count+1
    if count== 100:
        count = 0
        time.sleep(450)
    else:
        count = count+1
    
print(total_count)

天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
天猫链接
253


['https://chaoshi.tmall.com/']

In [133]:
dic ={'商品原始链接':shopping_links_list,
      '商品最终链接':shopping_final_links_list,
    '商品标题':title_list,
      '店铺名称':store_name_list,
      '店铺链接':store_link_list,
      '平台':platform_list,
      '是否下架':is_sold_out,
     '划线价':underline_price_list,
     '活动价':activity_price_list,
     '促销价':prmote_price_list,
     '活动信息':activity_info_list,
     '详细活动信息':activity_detail_info_list,
     '优惠券信息':coupon_info_list,
     '月销量':sales_list,
     '累计评论':comment_num_list}
tb_info_df = pd.DataFrame(dic)
# tb_info_df.to_csv('tb_data.csv')

In [130]:
print(len(activity_detail_info_list))
print(len(shopping_links_list))
print(len(shopping_final_links_list))
print(len(platform_list))
print(len(prmote_price_list))

80
80
80
80
80


In [135]:
tb_info_df.to_csv('version1.csv')

In [124]:
browser.get(links_df['Original Links'][19])
# browser.find_element_by_xpath('//*[@id="J_DetailMeta"]/div[1]/div[1]/div/div[2]/dl[3]/div[1]/dt').click()

In [125]:
links_df['Original Links'][19]

'https://item.taobao.com/item.htm?id=589069271255'

In [136]:
tb_info_df

,商品原始链接,商品最终链接,商品标题,店铺名称,店铺链接,平台,是否下架,划线价,活动价,促销价,活动信息,详细活动信息,优惠券信息,月销量,累计评论
0,https://item.taobao.com/item.htm?id=20565599246,https://item.taobao.com/item.htm?id=20565599246,野格力娇酒 野格圣鹿利口酒 JAGERMEIFTER德国原装进口 正品洋酒,179俱乐部,https://179club.taobao.com/,tb,0,138.00,,NaN,NaN,NaN,,,1127
1,https://item.taobao.com/item.htm?id=586375262300,https://detail.tmall.com/item.htm?id=586375262...,醉鹅娘 德国进口洋酒野格圣鹿酒利口酒力娇酒正品行货700ml,醉鹅娘酒类旗舰店,https:https://zuieniang.tmall.com/,tm,0,198.00,NaN,138.00,"满1件,送任意1件赠品（赠完即止）；满2件,送任意2件赠品（…","满1件 ,送任意1件赠品（赠完即止）\n满2件 ,送任意2件赠品（赠完即止）\n满3件 ,送...",送天猫积分69,7000+,50009
2,https://item.taobao.com/item.htm?id=611391648091,https://chaoshi.detail.tmall.com/item.htm?id=6...,【进口】网红野格圣鹿正品行货700ml+RIO锐澳微醺白桃味330ml*2瓶,天猫超市,https://chaoshi.tmall.com/,tm,0,,NaN,138.00,NaN,NaN,送天猫积分13,165,3086
3,https://item.taobao.com/item.htm?id=560263330969,https://detail.tmall.com/item.htm?id=560263330...,【选赠品】德国进口洋酒野格圣鹿利口酒700ml力娇酒 鹿头酒正品,WHISKY L！旗舰店,https:https://taishengyuanjl.tmall.com/,tm,0,198.00,NaN,138.00,"满1件,送任意1件赠品（赠完即止）；满2件,送任意2件赠品（…","满1件 ,送任意1件赠品（赠完即止）\n满2件 ,送任意2件赠品（赠完即止）\n满3件 ,送...",送天猫积分69,1743,23854
4,https://item.taobao.com/item.htm?id=595492174761,https://item.taobao.com/item.htm?id=595492174761,老宋的微醺23点 德国野格圣鹿利口酒700ml蓝鲸莫斯卡托香槟起泡酒,老宋的微醺23点,https://shop251938368.taobao.com/,tb,0,258.00,,168.00,超值优惠,NaN,淘金币可抵5.04元,340,4326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,https://item.taobao.com/item.htm?id=597885936127,https://item.taobao.com/item.htm?id=597885936127,洋酒 Ardbeg苏格兰进口阿贝雅柏阿德贝哥10年单一麦芽威士忌700ml,尊酒行,https://shop246764384.taobao.com/,tb,0,272.00,,NaN,NaN,NaN,,25,46
329,https://item.taobao.com/item.htm?id=542852633985,https://detail.tmall.com/item.htm?id=542852633985,郎家园洋酒ARDBEG 10Y.O 阿德贝哥雅柏10年单一麦芽威士忌酒700ml,郎家园酒类专营店,https:https://ljyjl.tmall.com/,tm,0,NaN,NaN,NaN,NaN,NaN,送天猫积分377,NaN,NaN
330,https://item.taobao.com/item.htm?id=569526201456,https://item.taobao.com/item.htm?id=569526201456,洋酒进口阿贝10年Ardbeg雅柏阿德贝哥单一麦芽苏格兰威士忌46%,麦子酒馆,https://shop110269456.taobao.com/,tb,0,336.00,,NaN,NaN,NaN,,3,36
331,https://item.taobao.com/item.htm?id=37358453689,https://item.taobao.com/item.htm?id=37358453689,正品行货洋酒 ARDBEG 雅柏艾雷 阿德贝哥/阿贝10年单一麦芽威士忌,广东龙粤酒行,https://longyjh.taobao.com/,tb,0,427.20,,340.00,促销活动,NaN,,14,50


In [ ]:
import datetime
import threading
 
def func():
  print("haha")
  #如果需要循环调用，就要添加以下方法
  timer = threading.Timer(86400, func)
  timer.start()
 
# 获取现在时间
now_time = datetime.datetime.now()
# 获取明天时间
next_time = now_time + datetime.timedelta(days=+1)
next_year = next_time.date().year
next_month = next_time.date().month
next_day = next_time.date().day
# 获取明天3点时间
next_time = datetime.datetime.strptime(str(next_year)+"-"+str(next_month)+"-"+str(next_day)+" 03:00:00", "%Y-%m-%d %H:%M:%S")
# # 获取昨天时间
# last_time = now_time + datetime.timedelta(days=-1)
 
# 获取距离明天3点时间，单位为秒
timer_start_time = (next_time - now_time).total_seconds()
print(timer_start_time)
# 54186.75975
 
#定时器,参数为(多少时间后执行，单位为秒，执行的方法)
timer = threading.Timer(timer_start_time, func)
timer.start()